### Collecting Training Data
In order to start collecting training data, you'll need to do the following:

* Enter Training Mode in the simulator.
* Start driving the car to get a feel for the controls.
* When you are ready, hit the record button in the top right to start recording.
* Continue driving for a few laps or till you feel like you have enough data.
* Hit the record button in the top right again to stop recording.

### Strategies for Collecting Data
Now that you have driven the simulator and know how to record data, it's time to think about collecting data that will ensure a successful model. There are a few general concepts to think about that we will later discuss in more detail:

* the car should stay in the center of the road as much as possible
* if the car veers off to the side, it should recover back to center
* driving counter-clockwise can help the model generalize
* flipping the images is a quick way to augment the data
* collecting data from the second track can also help generalize the model
* we want to avoid overfitting or underfitting when training the model
* knowing when to stop collecting more data




pip install --upgrade --ignore-installed setuptools

pip install python-socketio

python drive.py model.h5

python drive.py model.h5 run1

python video.py run1

python video.py run1 --fps 48


In [29]:
import csv
import cv2
import numpy as np
import os

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Conv2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

lines=[]
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
        
# print(lines)
images = []
measurements = []
for line in lines:
    for i in range(3):#center,left,right
        source_path = line[i]
#         filename = source_path.split('/')[-1]
        filename = source_path.split('\\')[-1]
        current_path = './data/IMG/'+filename
#         print(current_path)
#         print(os.path.exists(current_path))
        if(os.path.exists(current_path)):
            image = cv2.imread(current_path)
            images.append(image)
            measurement = float(line[3])
            measurements.append(measurement)

print(images[0].shape)
#Data augmentation        
augmented_images, augmented_measurements = [],[]
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)

print(augmented_images[0].shape)
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)
print(X_train.shape)
print(y_train.shape)

#Use Keras to train a network to do the following:
#1.Take in an image from the center camera of the car. This is the input to your neural network.
#2.Output a new steering angle for the car. 

model = Sequential()
# set up lambda layer
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))
#Cropping2D Layer
model.add(Cropping2D(cropping=((70,25),(0,0))))
# model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))

# model.add(Convolution2D(24,5,5, subsample=(2,2),activation="relu"))
# model.add(Convolution2D(36,5,5, subsample=(2,2),activation="relu"))
# model.add(Convolution2D(48,5,5, subsample=(2,2),activation="relu"))
# model.add(Convolution2D(64,3,3, activation="relu"))
# model.add(Convolution2D(64,3,3, activation="relu"))

model.add(Conv2D(24, (5, 5), strides=(2, 2), activation="relu"))
model.add(Conv2D(36, (5,5), strides=(2,2), activation="relu"))
model.add(Conv2D(48, (5,5), strides=(2,2), activation="relu"))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

#LeNet
# model.add(Convolution2D(6, 5, 5, activation="relu"))
# model.add(MaxPooling2D())
# model.add(Convolution2D(6, 5, 5, activation="relu"))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(120))
# model.add(Dense(84))
# model.add(Dense(1))

          
          
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save("model.h5")
          
          
          

./data/IMG/center_2020_01_03_13_35_56_900.jpg
True
(160, 320, 3)
1
./data/IMG/left_2020_01_03_13_35_56_900.jpg
True
(160, 320, 3)
2
./data/IMG/right_2020_01_03_13_35_56_900.jpg
True
(160, 320, 3)
3
./data/IMG/center_2020_01_03_13_35_56_967.jpg
True
(160, 320, 3)
4
./data/IMG/left_2020_01_03_13_35_56_967.jpg
True
(160, 320, 3)
5
./data/IMG/right_2020_01_03_13_35_56_967.jpg
True
(160, 320, 3)
6
./data/IMG/center_2020_01_03_13_35_57_035.jpg
True
(160, 320, 3)
7
./data/IMG/left_2020_01_03_13_35_57_035.jpg
True
(160, 320, 3)
8
./data/IMG/right_2020_01_03_13_35_57_035.jpg
True
(160, 320, 3)
9
./data/IMG/center_2020_01_03_13_35_57_103.jpg
True
(160, 320, 3)
10
./data/IMG/left_2020_01_03_13_35_57_103.jpg
True
(160, 320, 3)
11
./data/IMG/right_2020_01_03_13_35_57_103.jpg
True
(160, 320, 3)
12
./data/IMG/center_2020_01_03_13_35_57_170.jpg
True
(160, 320, 3)
13
./data/IMG/left_2020_01_03_13_35_57_170.jpg
True
(160, 320, 3)
14
./data/IMG/right_2020_01_03_13_35_57_170.jpg
True
(160, 320, 3)
15
./da

(160, 320, 3)
172
./data/IMG/left_2020_01_03_13_36_00_834.jpg
True
(160, 320, 3)
173
./data/IMG/right_2020_01_03_13_36_00_834.jpg
True
(160, 320, 3)
174
./data/IMG/center_2020_01_03_13_36_00_902.jpg
True
(160, 320, 3)
175
./data/IMG/left_2020_01_03_13_36_00_902.jpg
True
(160, 320, 3)
176
./data/IMG/right_2020_01_03_13_36_00_902.jpg
True
(160, 320, 3)
177
./data/IMG/center_2020_01_03_13_36_00_969.jpg
True
(160, 320, 3)
178
./data/IMG/left_2020_01_03_13_36_00_969.jpg
True
(160, 320, 3)
179
./data/IMG/right_2020_01_03_13_36_00_969.jpg
True
(160, 320, 3)
180
./data/IMG/center_2020_01_03_13_36_01_040.jpg
True
(160, 320, 3)
181
./data/IMG/left_2020_01_03_13_36_01_040.jpg
True
(160, 320, 3)
182
./data/IMG/right_2020_01_03_13_36_01_040.jpg
True
(160, 320, 3)
183
./data/IMG/center_2020_01_03_13_36_01_110.jpg
True
(160, 320, 3)
184
./data/IMG/left_2020_01_03_13_36_01_110.jpg
True
(160, 320, 3)
185
./data/IMG/right_2020_01_03_13_36_01_110.jpg
True
(160, 320, 3)
186
./data/IMG/center_2020_01_03_13

351
./data/IMG/center_2020_01_03_13_36_04_975.jpg
True
(160, 320, 3)
352
./data/IMG/left_2020_01_03_13_36_04_975.jpg
True
(160, 320, 3)
353
./data/IMG/right_2020_01_03_13_36_04_975.jpg
True
(160, 320, 3)
354
./data/IMG/center_2020_01_03_13_36_05_041.jpg
True
(160, 320, 3)
355
./data/IMG/left_2020_01_03_13_36_05_041.jpg
True
(160, 320, 3)
356
./data/IMG/right_2020_01_03_13_36_05_041.jpg
True
(160, 320, 3)
357
./data/IMG/center_2020_01_03_13_36_05_109.jpg
True
(160, 320, 3)
358
./data/IMG/left_2020_01_03_13_36_05_109.jpg
True
(160, 320, 3)
359
./data/IMG/right_2020_01_03_13_36_05_109.jpg
True
(160, 320, 3)
360
./data/IMG/center_2020_01_03_13_36_05_180.jpg
True
(160, 320, 3)
361
./data/IMG/left_2020_01_03_13_36_05_180.jpg
True
(160, 320, 3)
362
./data/IMG/right_2020_01_03_13_36_05_180.jpg
True
(160, 320, 3)
363
./data/IMG/center_2020_01_03_13_36_05_249.jpg
True
(160, 320, 3)
364
./data/IMG/left_2020_01_03_13_36_05_249.jpg
True
(160, 320, 3)
365
./data/IMG/right_2020_01_03_13_36_05_249.jpg

500
./data/IMG/right_2020_01_03_13_36_08_372.jpg
True
(160, 320, 3)
501
./data/IMG/center_2020_01_03_13_36_08_439.jpg
True
(160, 320, 3)
502
./data/IMG/left_2020_01_03_13_36_08_439.jpg
True
(160, 320, 3)
503
./data/IMG/right_2020_01_03_13_36_08_439.jpg
True
(160, 320, 3)
504
./data/IMG/center_2020_01_03_13_36_08_506.jpg
True
(160, 320, 3)
505
./data/IMG/left_2020_01_03_13_36_08_506.jpg
True
(160, 320, 3)
506
./data/IMG/right_2020_01_03_13_36_08_506.jpg
True
(160, 320, 3)
507
./data/IMG/center_2020_01_03_13_36_08_575.jpg
True
(160, 320, 3)
508
./data/IMG/left_2020_01_03_13_36_08_575.jpg
True
(160, 320, 3)
509
./data/IMG/right_2020_01_03_13_36_08_575.jpg
True
(160, 320, 3)
510
./data/IMG/center_2020_01_03_13_36_08_642.jpg
True
(160, 320, 3)
511
./data/IMG/left_2020_01_03_13_36_08_642.jpg
True
(160, 320, 3)
512
./data/IMG/right_2020_01_03_13_36_08_642.jpg
True
(160, 320, 3)
513
./data/IMG/center_2020_01_03_13_36_08_710.jpg
True
(160, 320, 3)
514
./data/IMG/left_2020_01_03_13_36_08_710.jpg

(160, 320, 3)
650
./data/IMG/right_2020_01_03_13_36_11_829.jpg
True
(160, 320, 3)
651
./data/IMG/center_2020_01_03_13_36_11_899.jpg
True
(160, 320, 3)
652
./data/IMG/left_2020_01_03_13_36_11_899.jpg
True
(160, 320, 3)
653
./data/IMG/right_2020_01_03_13_36_11_899.jpg
True
(160, 320, 3)
654
./data/IMG/center_2020_01_03_13_36_11_968.jpg
True
(160, 320, 3)
655
./data/IMG/left_2020_01_03_13_36_11_968.jpg
True
(160, 320, 3)
656
./data/IMG/right_2020_01_03_13_36_11_968.jpg
True
(160, 320, 3)
657
./data/IMG/center_2020_01_03_13_36_12_036.jpg
True
(160, 320, 3)
658
./data/IMG/left_2020_01_03_13_36_12_036.jpg
True
(160, 320, 3)
659
./data/IMG/right_2020_01_03_13_36_12_036.jpg
True
(160, 320, 3)
660
./data/IMG/center_2020_01_03_13_36_12_102.jpg
True
(160, 320, 3)
661
./data/IMG/left_2020_01_03_13_36_12_102.jpg
True
(160, 320, 3)
662
./data/IMG/right_2020_01_03_13_36_12_102.jpg
True
(160, 320, 3)
663
./data/IMG/center_2020_01_03_13_36_12_172.jpg
True
(160, 320, 3)
664
./data/IMG/left_2020_01_03_13

...

(160, 320, 3)
1466
./data/IMG/right_2020_01_03_13_36_30_611.jpg
True
(160, 320, 3)
1467
./data/IMG/center_2020_01_03_13_36_30_678.jpg
True
(160, 320, 3)
1468
./data/IMG/left_2020_01_03_13_36_30_678.jpg
True
(160, 320, 3)
1469
./data/IMG/right_2020_01_03_13_36_30_678.jpg
True
(160, 320, 3)
1470
./data/IMG/center_2020_01_03_13_36_30_746.jpg
True
(160, 320, 3)
1471
./data/IMG/left_2020_01_03_13_36_30_746.jpg
True
(160, 320, 3)
1472
./data/IMG/right_2020_01_03_13_36_30_746.jpg
True
(160, 320, 3)
1473
./data/IMG/center_2020_01_03_13_36_30_819.jpg
True
(160, 320, 3)
1474
./data/IMG/left_2020_01_03_13_36_30_819.jpg
True
(160, 320, 3)
1475
./data/IMG/right_2020_01_03_13_36_30_819.jpg
True
(160, 320, 3)
1476
./data/IMG/center_2020_01_03_13_36_30_887.jpg
True
(160, 320, 3)
1477
./data/IMG/left_2020_01_03_13_36_30_887.jpg
True
(160, 320, 3)
1478
./data/IMG/right_2020_01_03_13_36_30_887.jpg
True
(160, 320, 3)
1479
./data/IMG/center_2020_01_03_13_36_30_957.jpg
True
(160, 320, 3)
1480
./data/IMG/lef

(160, 320, 3)
1624
./data/IMG/left_2020_01_03_13_36_34_244.jpg
True
(160, 320, 3)
1625
./data/IMG/right_2020_01_03_13_36_34_244.jpg
True
(160, 320, 3)
1626
./data/IMG/center_2020_01_03_13_36_34_311.jpg
True
(160, 320, 3)
1627
./data/IMG/left_2020_01_03_13_36_34_311.jpg
True
(160, 320, 3)
1628
./data/IMG/right_2020_01_03_13_36_34_311.jpg
True
(160, 320, 3)
1629
./data/IMG/center_2020_01_03_13_36_34_380.jpg
True
(160, 320, 3)
1630
./data/IMG/left_2020_01_03_13_36_34_380.jpg
True
(160, 320, 3)
1631
./data/IMG/right_2020_01_03_13_36_34_380.jpg
True
(160, 320, 3)
1632
./data/IMG/center_2020_01_03_13_36_34_451.jpg
True
(160, 320, 3)
1633
./data/IMG/left_2020_01_03_13_36_34_451.jpg
True
(160, 320, 3)
1634
./data/IMG/right_2020_01_03_13_36_34_451.jpg
True
(160, 320, 3)
1635
./data/IMG/center_2020_01_03_13_36_34_523.jpg
True
(160, 320, 3)
1636
./data/IMG/left_2020_01_03_13_36_34_523.jpg
True
(160, 320, 3)
1637
./data/IMG/right_2020_01_03_13_36_34_523.jpg
True
(160, 320, 3)
1638
./data/IMG/cente

True
(160, 320, 3)
1774
./data/IMG/left_2020_01_03_13_36_37_690.jpg
True
(160, 320, 3)
1775
./data/IMG/right_2020_01_03_13_36_37_690.jpg
True
(160, 320, 3)
1776
./data/IMG/center_2020_01_03_13_36_37_757.jpg
True
(160, 320, 3)
1777
./data/IMG/left_2020_01_03_13_36_37_757.jpg
True
(160, 320, 3)
1778
./data/IMG/right_2020_01_03_13_36_37_757.jpg
True
(160, 320, 3)
1779
./data/IMG/center_2020_01_03_13_36_37_826.jpg
True
(160, 320, 3)
1780
./data/IMG/left_2020_01_03_13_36_37_826.jpg
True
(160, 320, 3)
1781
./data/IMG/right_2020_01_03_13_36_37_826.jpg
True
(160, 320, 3)
1782
./data/IMG/center_2020_01_03_13_36_37_898.jpg
True
(160, 320, 3)
1783
./data/IMG/left_2020_01_03_13_36_37_898.jpg
True
(160, 320, 3)
1784
./data/IMG/right_2020_01_03_13_36_37_898.jpg
True
(160, 320, 3)
1785
./data/IMG/center_2020_01_03_13_36_37_964.jpg
True
(160, 320, 3)
1786
./data/IMG/left_2020_01_03_13_36_37_964.jpg
True
(160, 320, 3)
1787
./data/IMG/right_2020_01_03_13_36_37_964.jpg
True
(160, 320, 3)
1788
./data/IMG/

(160, 320, 3)
1934
./data/IMG/right_2020_01_03_13_36_41_339.jpg
True
(160, 320, 3)
1935
./data/IMG/center_2020_01_03_13_36_41_406.jpg
True
(160, 320, 3)
1936
./data/IMG/left_2020_01_03_13_36_41_406.jpg
True
(160, 320, 3)
1937
./data/IMG/right_2020_01_03_13_36_41_406.jpg
True
(160, 320, 3)
1938
./data/IMG/center_2020_01_03_13_36_41_474.jpg
True
(160, 320, 3)
1939
./data/IMG/left_2020_01_03_13_36_41_474.jpg
True
(160, 320, 3)
1940
./data/IMG/right_2020_01_03_13_36_41_474.jpg
True
(160, 320, 3)
1941
./data/IMG/center_2020_01_03_13_36_41_541.jpg
True
(160, 320, 3)
1942
./data/IMG/left_2020_01_03_13_36_41_541.jpg
True
(160, 320, 3)
1943
./data/IMG/right_2020_01_03_13_36_41_541.jpg
True
(160, 320, 3)
1944
./data/IMG/center_2020_01_03_13_36_41_610.jpg
True
(160, 320, 3)
1945
./data/IMG/left_2020_01_03_13_36_41_610.jpg
True
(160, 320, 3)
1946
./data/IMG/right_2020_01_03_13_36_41_610.jpg
True
(160, 320, 3)
1947
./data/IMG/center_2020_01_03_13_36_41_679.jpg
True
(160, 320, 3)
1948
./data/IMG/lef

(160, 320, 3)
(4170, 160, 320, 3)
(4170,)
Instructions for updating:
Use tf.cast instead.
Train on 3336 samples, validate on 834 samples
Epoch 1/5
3336/3336 [==============================] - 29s 9ms/step - loss: 0.0159 - val_loss: 0.1141
Epoch 2/5
3336/3336 [==============================] - 23s 7ms/step - loss: 0.0134 - val_loss: 0.1058
Epoch 3/5
3336/3336 [==============================] - 23s 7ms/step - loss: 0.0140 - val_loss: 0.0997
Epoch 4/5
3336/3336 [==============================] - 23s 7ms/step - loss: 0.0133 - val_loss: 0.1087
Epoch 5/5
3336/3336 [==============================] - 24s 7ms/step - loss: 0.0125 - val_loss: 0.1042
